## 2.75 Machine Learning for Ranking using scikit learn - Exercises

We are going to be using [scikit learn](http://scikit-learn.org/stable/index.html)

<img src='files/resources/scikit-learn-logo-small.png' align='left'><h2>Machine Learning in Python</h2>

<br>
* Simple and efficient tools for data mining and data analysis
* Accessible to everybody, and reusable in various contexts
* Built on NumPy, SciPy, and matplotlib
* Open source, commercially usable - BSD license


## Data

In [1]:
import os 
import pandas as pd

In [2]:
data = pd.read_csv("data/fullDataset.tsv", sep="\t",header=0)

/home/otter/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data[['query','Title','label_relevanceBinary','feature_1','feature_9']].head()

,query,Title,label_relevanceBinary,feature_1,feature_9
0,all star ticket 2015,6N/7D Disney World All Star Music~ All Inclusi...,0,2676.618000,139
1,mga top bows,"MGTC/TD/TF, MGA, Morris Minor Top Wing Bolts s...",0,2676.618000,208
2,cross document markers,New Cross Vice Gel Ink Pen Gift Set & Document...,1,2676.618000,226
3,wall cabinet white,White PAPER TOWEL ROLL HOLDER Cabinet Wall Mou...,0,1557.416626,158
4,2.5 jato cooling head,ENGINE COOLING HEAD TRAXXAS JATO TRX 2.5 NITRO...,1,2676.618000,150


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78500 entries, 0 to 78499
Data columns (total 19 columns):
key                      78500 non-null int64
query                    78500 non-null object
Title                    78494 non-null object
LeafCats                 78500 non-null object
ItemID                   78500 non-null int64
X_unit_id                78500 non-null object
SCORE                    78500 non-null object
label_relevanceGrade     78500 non-null int64
label_relevanceBinary    78500 non-null float64
feature_1                78500 non-null float64
feature_2                78500 non-null float64
feature_3                78500 non-null float64
feature_4                78500 non-null float64
feature_5                78500 non-null float64
feature_6                78500 non-null float64
feature_7                78500 non-null float64
feature_8                78500 non-null float64
feature_9                78500 non-null int64
feature_10               78499 non-null 

In [6]:
data.describe()

,key,ItemID,label_relevanceGrade,label_relevanceBinary,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
count,78500.000000,7.850000e+04,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78500.000000,78499.000000
mean,98346.614064,4.869803e+04,5.254777,0.713653,2821.519304,0.562437,-7.754530,-4.280808,47.526718,0.439879,50746.223210,-240684.450178,171.255248,-62.985598
std,70253.651231,2.890059e+06,1.334406,8.446960,834.067146,0.497410,1.162798,0.772099,24.222248,18.439044,228258.768038,527649.016958,41.412263,48.710378
min,1.000000,1.000000e+00,0.000000,0.000000,-1890.000000,-9.658937,-13.563713,-10.324562,0.000000,0.000000,-1000000.000000,-1000000.000000,-100.000000,-100.000000
25%,39209.750000,1.908475e+04,5.000000,0.000000,2676.618000,0.000000,-8.285197,-4.736623,30.806008,0.101174,0.000000,-1000000.000000,144.000000,-100.000000
50%,78577.500000,3.831350e+04,6.000000,1.000000,2676.618000,1.000000,-7.651813,-4.327543,44.189650,0.365236,191.000000,0.000000,172.000000,-100.000000
75%,157546.250000,5.765925e+04,6.000000,1.000000,3237.216614,1.000000,-7.390667,-3.766841,60.076828,0.613038,4707.250000,98130.000000,203.000000,0.620428
max,234520.000000,8.097468e+08,6.000000,2363.750000,5286.000000,1.000000,-2.726150,42.595760,273.239624,5166.000000,3339990.000000,1000000.000000,240.000000,10.940677


## Data Exploration

We will start by exploring the data - however first we remove all the records that contain missing data.  Generally machine learning algorithms require complete records - some types of classification and regression tree's are designed to deal with missing data - but in general we have to deal with it ourselves during preprocessing.

In [7]:
data = data.dropna()
data.describe()

,key,ItemID,label_relevanceGrade,label_relevanceBinary,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
count,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.000000,78493.00000,78493.000000,78493.000000
mean,98353.149287,38385.733390,5.254774,0.683526,2821.567172,0.562598,-7.754471,-4.281164,47.528829,0.374066,50762.626642,-240647.46213,171.256023,-62.984023
std,70253.061135,22259.156287,1.334363,0.465103,834.042432,0.496069,1.162636,0.753205,24.222072,0.280593,228237.868119,527631.90141,41.400772,48.710849
min,1.000000,1.000000,0.000000,0.000000,-1890.000000,0.000000,-13.563713,-10.324562,0.000000,0.000000,0.000000,-1000000.00000,0.000000,-100.000000
25%,39212.000000,19088.000000,5.000000,0.000000,2676.618000,0.000000,-8.285108,-4.736623,30.808287,0.101141,0.000000,-1000000.00000,144.000000,-100.000000
50%,78584.000000,38316.000000,6.000000,1.000000,2676.618000,1.000000,-7.651790,-4.327510,44.190533,0.365175,191.000000,0.00000,172.000000,-100.000000
75%,157548.000000,57660.000000,6.000000,1.000000,3237.285645,1.000000,-7.390667,-3.766839,60.078297,0.613038,4706.000000,98130.00000,203.000000,0.620932
max,234520.000000,77080.000000,6.000000,1.000000,5286.000000,1.000000,-2.726150,-1.106641,273.239624,1.000000,3339990.000000,1000000.00000,240.000000,10.940677


In [8]:
from bokeh.charts import defaults, Bar, output_notebook, show
defaults.plot_width = 500
defaults.plot_height = 300
output_notebook(hide_banner=True)

We can see that low relevance results are under represented:

In [9]:
p = Bar(data, 'label_relevanceGrade',  values='label_relevanceGrade', agg='count', title="Distribution of Relevance Grade")
show(p)

For machine learning a lazy algorithm could predict grade 6 for everything.  Predicting the rarer class 0 or 1 has less payoff.  To 'encourage' our algorithm to more discriminating classification we can turn the 6-level relevance score into a binary target.  Now it does not pay to classify everything as relevant.

In [10]:
p = Bar(data, 'label_relevanceBinary',  values='label_relevanceBinary', agg='count', title="Distribution of Relevance Grade")
show(p)

Let's look at the relationship between this binary relevance target and the features.

In [11]:
from bokeh.io import gridplot
from bokeh.charts import BoxPlot, output_file, show

defaults.plot_width = 400
defaults.plot_height = 300

def bplot(feature):
    return BoxPlot(data, values=feature, label="label_relevanceBinary", outliers=False)

p = gridplot ( [ 
        [ bplot("feature_1"), bplot("feature_2") ],
        [ bplot("feature_3"), bplot("feature_4") ],
        [ bplot("feature_5"), bplot("feature_6") ],
        [ bplot("feature_7"), bplot("feature_8") ],
        [ bplot("feature_9"), bplot("feature_10") ] ] ) 
show(p)

## Training / Test data

We also want to scale our features to have zero mean and unit standard deviation.  This avoids certain features dominating the machine learning objective function - something that can lead to poorly performing models.

To reduce data size for our virtual machine I take a 20k sample of the data - I also rebalance the data ensuring equal representation of relevant and not-relevant labels.  This simplifies our model accuracy comparison during class.

In [14]:
from sklearn import preprocessing
from sklearn.externals import joblib

scaler = preprocessing.StandardScaler().fit(data.loc[:,"feature_1":"feature_10"])

# Serialize for later when we deploy the model
joblib.dump(scaler,'models/scaler.pkl')

data_0 = data.loc[(data.label_relevanceBinary == 0),]
data_1 = data.loc[(data.label_relevanceBinary == 1),]

data_0 = data_0.sample(10000, replace=True)
data_1 = data_1.sample(10000, replace=True)

data_balanced = data_0.append(data_1)

y = data_balanced["label_relevanceBinary"].values

X = scaler.transform(data_balanced.loc[:,"feature_1":"feature_10"])

features = data_balanced.loc[:,"feature_1":"feature_10"].columns

print(X.mean(axis=0))
print(X.std(axis=0))

[-0.10009745 -0.07871521 -0.00046007 -0.06528945 -0.02684132 -0.09875913
 -0.02629727  0.00358263  0.01025657 -0.02010531]
[ 0.98810454  1.00681173  0.97204962  1.01149013  1.00494525  0.99607078
  0.91933897  0.9823597   0.98532499  0.99354431]


When machine learning we train on one data set and measure performance on another - this ensures that our performance metrics are representative of the actual performance of the model - this helps avoid overfitting.

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=73)

print("X training : " + str(X_train.shape))
print("y training : " + str(y_train.shape))
print("X test     : " + str(X_test.shape))
print("y test     : " + str(y_test.shape))

X training : (15000, 10)
y training : (15000,)
X test     : (5000, 10)
y test     : (5000,)


## Baseline Model

Model 01 - just predict everything to be the dominant class and check performance - this is our baseline.

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'>It is a good idea to label your models.  
Keeping track of what worked and did not work is much easier this way.

There is no model to fit for our baseline case - we can create the predictions directly:

In [17]:
y_pred_01 = [1] * y_test.shape[0]

sklearn includes the metrics class which has many methods to compute performance metrics for a model.  Here we use `confusion_matrix()`, `classification_report()` and `accuracy_score()` methods:

In [18]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print('Confusion Matrix:\n'+str(confusion_matrix(y_test, y_pred_01)))
print('\nClassification Report:\n'+str(classification_report(y_test, y_pred_01)))
print('Accuracy = {0:6.2f}'.format(accuracy_score(y_test, y_pred_01)))

Confusion Matrix:
[[   0 2483]
 [   0 2517]]

Classification Report:
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      2483
        1.0       0.50      1.00      0.67      2517

avg / total       0.25      0.50      0.34      5000

Accuracy =   0.50


/home/otter/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


We need to define the metric we are going to use to evaluate and compare models.  There are many classification metrics - based on the cells in the confusion matrix:
 
|                  || *Predicted*  | *Class*   ||
|------------------||--------------------------|||
|                  ||     -    |    +    | Total |
|*True*   |   -     |   TN     |    FP   |  N    |
|*Classs* |   +     |   FN     |    TP   |  P    |
|         | Total   |   N*     |    P*   |       |


Now we can define metrics based on the contingency table:

| Name                      |   Definition  |   Synonyms                                  |
|---------------------------|---------------|---------------------------------------------|
| False Positive Rate       |   FP / N      | Type-I error, 1-Specificity                 |
| True Positive Rate        |   TP / P      | 1-Type II error, power, sensitivity, recall |
| Positive Predictive Value |   TP / P*     | Precision, 1-false discovery proportion     |
| Negative Predictive Value |   TN / N*     |                                             |

A good diagram (shown below) and more discussion is available on [wikipedia](https://en.wikipedia.org/wiki/Precision_and_recall)
<img src='files/resources/Precisionrecall.png' align='center'>

Sometimes we talk about the accuracy of a model as:

$
accuracy = \frac{(TP + TN)}{( N + P )}
$

Generally we are interested in precision and recall.  

* Precision
    * The fraction of cases predicted to be positive that are actually positive
    * High precision is acheived by ensuring that all the predicted positive cases are actually positive
    * Quality
* Recall
    * The fraction of the positive cases predicted to be positive 
    * High recall is acheived by a model predicts as many of the positive cases as possible 
    * Quantity

We can also define classification metrics that compbine both precision and recall.  The F1-score is one such metric that computes the harmonic mean of precision and recall - applying equal emphasis to precision as recall:

$
F_1 = 2 \times \frac{ precision . recall}{precision + recall} 
$

But we may choose to attach emphasis to recall:

$
F_2 = 5 \times \frac{ precision . recall}{ (4 . precision) + recall} 
$

Or we may choose to place higher emphasis on precision:

$
F_{0.5} = 1.25 \times \frac{ precision . recall}{( 0.25 . precision) + recall} 
$

For this exercise we are going to focus on the accuracy score.

We can see that recall is 100% - we predicted every value that was positive correctly - but precision is low - of those predicted to be positve only 50% were actually postive.  This yields an F1 score of 81% and an accuracy of 50%.  This is our baseline upon which we want to improve.

## Class Exercise

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>Your goal is to improve on this model.  The class is going to compete inorder to find the model with the best test accuracy.   
Here I provide a long list of the models to explore - you will need to search through hyperparameters and features.

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'>Your results must be reproducible by others!!  
So when you beat the class best shout out and we will try and reproduce the results!

Models to experiment with:

```
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=?).

from sklearn import linear_model
clf = linear_model.LogisticRegression(penalty=?)

from sklearn import svm
clf = svm.SVC(kernel='linear', C=?)

from sklearn import svm
clf = svm.SVC(kernel='rbf', C=?)

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_leaf=?, max_depth =?)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=?)

from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=?, learning_rate=?, max_depth=?)

```
See documentation [here](http://scikit-learn.org/stable/index.html)